In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customer-support-on-twitter/sample.csv
/kaggle/input/customer-support-on-twitter/twcs/twcs.csv


## Imports

In [12]:
import re
import nltk
import spacy
import string
full_df = pd.read_csv("../input/customer-support-on-twitter/twcs/twcs.csv", nrows=10)
full_df["text"]=full_df["text"].astype(str)
full_df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


## Data Cleaning

In [13]:
for x in list(full_df.columns):
    if x not in ['tweet_id','text']:
        del full_df[x]
df=full_df
ps = list(";?.:!,")

for p in ps:   
    df['text'] = df['text'].str.replace(p, '')
    
df['text'] = df['text'].str.replace("    ", " ")
df['text'] = df['text'].str.replace('"', '')
df['text'] = df['text'].apply(lambda x: x.replace('\t', ' '))
df['text'] = df['text'].str.replace("'s", "")
df['text'] = df['text'].apply(lambda x: x.replace('\n', ' '))
df['text'] = df['text'].apply(lambda x:x.lower())
df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


,tweet_id,text
0,1,@115712 i understand i would like to assist yo...
1,2,@sprintcare and how do you propose we do that
2,3,@sprintcare i have sent several private messag...
3,4,@115712 please send us a private message so th...
4,5,@sprintcare i did
5,6,@115712 can you please send us a private messa...
6,8,@sprintcare is the worst customer service
7,11,@115713 this is saddening to hear please shoot...
8,12,@sprintcare you gonna magically change your co...
9,15,@115713 we understand your concerns and we'd l...


In [14]:
def cleandata(txt):
    txt = txt.lower()
    txt = re.sub('[^a-zA-Z]', ' ', txt)
    txt = re.sub('http\S+\s*', ' ', txt)
    txt = re.sub('RT|cc', ' ', txt)
    txt = re.sub('#\S+', '', txt) 
    txt = re.sub('@\S+', '  ', txt) 
    txt = re.sub('\s+', ' ', txt) 
    return txt
df['cleaned data'] = df['text'].apply(lambda x: cleandata(x))
del df['text']

In [15]:
df

,tweet_id,cleaned data
0,1,i understand i would like to assist you we wo...
1,2,sprintcare and how do you propose we do that
2,3,sprintcare i have sent several private messag...
3,4,please send us a private message so that we c...
4,5,sprintcare i did
5,6,can you please send us a private message so t...
6,8,sprintcare is the worst customer service
7,11,this is saddening to hear please shoot us a d...
8,12,sprintcare you gonna magically change your co...
9,15,we understand your concerns and we d like for...


## Bag of Words for all 10 rows, w/o stopword removal

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
CountVec = CountVectorizer(ngram_range=(1,1),
                           stop_words=None)
#transform
Count_data = CountVec.fit_transform(df['cleaned data'])
 
#create dataframe
cv_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())
cv_dataframe

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aa,about,and,as,assist,at,can,change,click,concerns,...,top,understand,us,usual,we,whole,worst,would,you,your
0,0,0,0,0,2,0,0,0,0,0,...,0,1,0,0,1,0,0,2,2,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,0,0,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,1,1,1,0,1,0,...,1,0,1,0,1,0,0,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,0,0,2,0,0,0,...,0,0,1,0,0,0,0,0,1,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
8,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,1
9,1,0,1,0,1,0,1,0,0,1,...,0,1,1,0,3,0,0,0,2,1


In [17]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

df['filtered text'] = df['cleaned data'].apply(remove_stopwords)
del df['cleaned data']
df

,tweet_id,filtered text
0,1,understand would like assist would need get pr...
1,2,sprintcare propose
2,3,sprintcare sent several private messages one r...
3,4,please send us private message assist click me...
4,5,sprintcare
5,6,please send us private message gain details ount
6,8,sprintcare worst customer service
7,11,saddening hear please shoot us dm look kc
8,12,sprintcare gon na magically change connectivit...
9,15,understand concerns like please send us direct...


## Bag of Words for all 10 rows, after stopword removal

In [18]:
Count_data = CountVec.fit_transform(df['filtered text'])
 
#create dataframe
cv_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())
cv_dataframe

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aa,assist,change,click,concerns,connectivity,customer,details,direct,dm,...,several,shoot,sprintcare,top,understand,us,usual,whole,worst,would
0,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
7,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,1,0,0,0,0
8,0,0,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
9,1,1,0,0,1,0,0,0,1,0,...,0,0,0,0,1,1,0,0,0,0


In [19]:
stop_words = set(stopwords.words('english'))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r